# Temperature Analysis 2: Bonus

In [1]:
# Import dependencies for matplotlib
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
# Import other dependencies
import numpy as np
import pandas as pd
import datetime as dt

In [7]:
# Import SQL Alchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [8]:
# Create the engine to the Hawaii SQLite Database
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [9]:
# Create a base and prepare the base
Base = automap_base()
Base.prepare(engine, reflect = True)

In [10]:
# See the classes in the database
Base.classes.keys()

['measurement', 'station']

In [12]:
# Save a reference to the tables
Station = Base.classes.station
Measurement = Base.classes.measurement

In [13]:
# Start a session
session = Session(engine)

In [14]:
# Function from provided starter code:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]
